##Simple Object Detection in TensorFlow

Walk through of how to use object detection models available in the TensorFlow Hub

* Explore the tensorflow hub for object detection models
* Load models into the workspace
* Preprocess an image for inference 
* Run inference on the models and inpect the output

##Imports

In [11]:
import tensorflow as tf
import tensorflow_hub as hub
from PIL import ImageOps, Image
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

#Download the model from the hub

Tensorflow hub is a repository of trained machine learning models which can be reused in projects 

In [2]:
#Inception ResNet V2
module_handle = 'https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1'

##Load the model 

In [3]:
model = hub.load(module_handle)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


##Choose the default signature
Some models in the tensorflow hub can be used for different tasks. Each model's documentation should show what signature to use when running the model 

In [5]:
detector = model.signatures['default']

##Download and resize the images

`download_and_resize_image` function downloads an image specified by a given url, pre-processes it, and saves it to the disk

In [12]:
def download_and_resize_image(url, new_width=256, new_height=256):
  """
  Fetches an image online, resizes it and saves it locally.

  Args:
    url (string) -- link to the image
    new_width (int) -- size in pixels used for resizing the width of the image 
    new_height (int) -- size in pixels used for resizing the height of the image

  Returns: 
    (string) -- path to saved image 
  """

  #Create a temporary file ending with .jpg
  _, filename = tempfile.mkstemp(suffix='.jpg')

  #Opens the given url 
  response = urlopen(url)

  #Reads the image fetched from the url
  image_data = response.read()

  #Puts the image data in memory buffer
  image_data = BytesIO(image_data)

  #Opens the image
  pil_image = Image.open(image_data)

  #Resizes the image and will crop if aspect ratio is different 
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)

  #Converts to the RGB colorspace
  pil_image_rgb = pil_image.convert('RGB')

  #Saves the image to the temporay file created earlier
  pil_image_rgb.save(filename, format='JPEG', quality=90)

  print("Image downloaded to %s." % filename)

  return filename

##Download and Preprocess an image

Use the `download_and_resize_image` utility to get a sample image online and save it locally



In [13]:
#Feel free to try different URL's for the image 
image_url = "https://upload.wikimedia.org/wikipedia/commons/f/fb/20130807_dublin014.JPG"

downloaded_image_path = download_and_resize_image(image_url, 3872, 2592)

Image downloaded to /tmp/tmplwd5eyu2.jpg.


##Run Detector

`run_detector` will take in the object detection model `detector` and the path to a sample image, then use this model to detect objects and display its predicted class categotries and detection boxes

* run_detector uses `load_image` to convert the image to a tensor

In [14]:
def load_img(path):
  """
  Loads a JPEG image and converts it to a tensor

  Args:
    path (string): path to a locally saved JPEG image

  Returns:
    (tensor): an image tensor
  """

  #Read the file 
  img = tf.io.read_file(path)

  #Convert to tensor
  img = tf.image.decode_jpeg(img, channels=3)

  return img


def run_detector(detector, path):
  """
  Runs inference on a local file using an object detection model 

  Args:
    detector (model): an object detection model loaded from TF Hub
    path (string): path to an image saved locally

  """

  #Load an image tensor from a local file path
  img = load_img(path)

  #add a batch dimension in front of the tensor 
  converted_img = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]

  #run inference using the model 
  result = detector(converted_img)

  #Save the results in a dictionary 
  result = {key: value.numpy() for key, value in result.items()}

  #print results 
  print('Found %d objects.' % len(result['detection_scores']))

  print(result['detection_scores'])
  print(result['detection_class_entities'])
  print(result['detection_boxes'])

##Run inference on the image

Run the detector by calling the `run_detector` function. This will print the number of objects found follow by three lists 

* The detection scores of each object found (ie how confident the model is)
* The classes of each object found 
* The bounding box coordinates of each object

In [15]:
#Runs the object detection model and prints info about objects found
run_detector(detector, downloaded_image_path)

Found 100 objects.
[0.6532176  0.6105047  0.60152537 0.59255654 0.5917778  0.5815494
 0.5505322  0.4957557  0.47424877 0.47322154 0.44066462 0.40511453
 0.39803782 0.39406568 0.37148535 0.36155996 0.3615074  0.34689203
 0.33362305 0.31252864 0.2887799  0.25758374 0.25748894 0.25196055
 0.24782118 0.23412395 0.20432054 0.20324677 0.17988306 0.17964779
 0.17374504 0.1643149  0.16031115 0.15895294 0.15620399 0.15468778
 0.1475446  0.13622598 0.12740147 0.12555622 0.12102702 0.11813023
 0.11387318 0.11229111 0.11129158 0.09718707 0.09137256 0.08976004
 0.08880235 0.08633568 0.08337442 0.08095161 0.07988821 0.07741437
 0.0773216  0.07631022 0.07507849 0.07386065 0.07233308 0.07204039
 0.07110174 0.06935585 0.06825589 0.06427862 0.06248555 0.06226387
 0.06211162 0.05940023 0.05798782 0.05784458 0.05725544 0.05346682
 0.05304347 0.05242869 0.04892596 0.04813037 0.0457847  0.04424177
 0.04337562 0.04277798 0.04262635 0.04163505 0.04081763 0.03976004
 0.03945957 0.03944291 0.03863972 0.03769058